In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import warnings

/Users/ironman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1.- Plant Functional Type Prediction: Can you train a machine learning model to predict the Plant Functional Type (PFT) for each individual tree in order to accurately predict the plot-level PFT distribution? Apply your model to the TLS (Terrestrial Laser Scanning) dataset to enrich the current TLS-derived tree lists with PFT labels.


In [2]:
fia_species = pd.read_csv("TLS/FIATreeSpeciesCode_pft.csv",sep=";")

In [3]:
fia_species.columns

Index(['SPCD', 'COMMON_NAME', 'SCI_NAME', 'SPGRPCD.E', 'SPGRPCD.W', 'MAJGRP',
       'OCC.NC', 'OCC.NE', 'OCC.PNW', 'OCC.RM', 'OCC.SO', 'PFT'],
      dtype='object')

In [4]:
fia_species

,SPCD,COMMON_NAME,SCI_NAME,SPGRPCD.E,SPGRPCD.W,MAJGRP,OCC.NC,OCC.NE,OCC.PNW,OCC.RM,OCC.SO,PFT
0,10,fir spp.,Abies spp,6,12,2,X,X,-,-,X,Evergreen conifer
1,11,Pacific silver fir,Abies amabilis,9,12,2,-,-,X,-,-,Evergreen conifer
2,12,balsam fir,Abies balsamea,6,12,2,X,X,-,-,X,Evergreen conifer
3,14,Santa Lucia or bristlecone fir,Abies bracteata,9,12,2,-,-,X,-,-,Evergreen conifer
4,15,white fir,Abies concolor,9,12,2,X,-,X,X,X,Evergreen conifer
...,...,...,...,...,...,...,...,...,...,...,...,...
453,995,tungoil tree,Vernicia fordii,43,47,4,-,-,-,-,X,Deciduous
454,996,smoketree,Cotinus obovatus,43,47,4,X,-,-,-,X,Deciduous
455,997,Russian-olive,Elaeagnus angustifolia,43,47,3,X,X,X,-,X,Nitrogen fixing legumes
456,998,Unknown dead hardwood,Tree broadleaf,43,47,3,X,X,X,-,X,Broadleaf


In [5]:
fia_species["PFT"].value_counts()

PFT
Deciduous broadleaf              120
Evergreen conifer                112
Deciduous                         99
Evergreen                         25
Evergreen broadleaf               23
Flowering plants                  18
Deciduous shrubs                  14
Palm                               9
Nitrogen fixing legumes            8
Shrub                              8
Deciduous confier                  8
Mangrove                           4
Deciduous and evergreen trees      2
Broadleaf                          1
Monocot                            1
Deciduous gymnosperm               1
Evergreen                          1
Wetland tree                       1
Tree                               1
Deciduous hardwood                 1
Unknown                            1
Name: count, dtype: int64

In [6]:
tls=pd.read_csv("TLS/TLS_treelist.csv")

In [7]:
tls

,Unnamed: 0,TreeID,X,Y,Radius,Error,H,h_radius,DBH,BasalA,plot_blk
0,0,1,10.673382,2.127033,0.266540,0.001199,12.560,1.3,20.987379,2.402324,CAFKU_0101_20240802_1
1,3,6,2.429448,1.055521,0.166444,0.000730,8.742,1.3,13.105771,0.936786,CAFKU_0101_20240802_1
2,4,8,2.149436,0.950025,0.759981,0.001151,13.469,1.3,59.840890,19.530404,CAFKU_0101_20240802_1
3,5,9,1.962382,8.781092,0.267219,0.001217,18.144,1.3,21.040854,2.414582,CAFKU_0101_20240802_1
4,6,12,1.433234,0.234223,0.208154,0.000543,13.790,1.3,16.390056,1.465130,CAFKU_0101_20240802_1
...,...,...,...,...,...,...,...,...,...,...,...
1377,17,21,-9.820196,2.799561,0.220015,0.000291,30.631,1.3,17.323968,1.636854,CATCU_0016_20240929_1
1378,18,22,2.043417,-9.313768,0.308002,0.000450,23.077,1.3,24.252048,3.207835,CATCU_0016_20240929_1
1379,20,24,-8.147035,6.844047,0.226510,0.000211,30.127,1.3,17.835430,1.734931,CATCU_0016_20240929_1
1380,21,25,-2.692303,-8.172273,0.180451,0.000266,29.300,1.3,14.208700,1.101093,CATCU_0016_20240929_1


In [8]:
blk=pd.read_csv("TLS/blk_plot_identification.csv")

In [9]:
blk

,Unnamed: 0,inventory_id,site_project,site_project_label,site_name,site_name_label,inventory_pre_post_fire,inventory_pre_post_fire_label,plot_num,plot_blk,inventory_date,inventory_hour,plot_coord_x,plot_coord_y,plot_coord_srs,plot_declination,inventory_plot_diam,plot_slopeper,plot_aspectdegre
0,0,143_pre_SHA_burn3d,burn3d,Burnpro3d,SHA,Shaver Lake (SHA),pre,Prefire,143,CAFKU_0143_20240721_1,7/21/24,14:44,297590,4108796,26911,12.28 east,11.3,14,261
1,1,140_pre_SHA_burn3d,burn3d,Burnpro3d,SHA,Shaver Lake (SHA),pre,Prefire,140,CAFKU_0140_20240721_1,7/21/24,11:03,297640,4108747,26911,12.28 east,11.3,12,307
2,2,152_pre_SHA_burn3d,burn3d,Burnpro3d,SHA,Shaver Lake (SHA),pre,Prefire,152,CAFKU_0152_20240721_1,7/21/24,8:25,297694,4108903,26911,12.28 east,11.3,19,319
3,3,147_pre_SHA_burn3d,burn3d,Burnpro3d,SHA,Shaver Lake (SHA),pre,Prefire,147,CAFKU_0147_20240720_1,7/20/24,14:04,297646,4108845,26911,12.28 east,11.3,15,333
4,4,151_pre_SHA_burn3d,burn3d,Burnpro3d,SHA,Shaver Lake (SHA),pre,Prefire,151,CAFKU_0151_20240720_1,7/20/24,10:39,297603,4108893,26911,12.28 east,11.3,14,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,158,40_pre_puc_burn3d,burn3d,Burnpro3d,puc,Pacific Union College (PUC),pre,Prefire,40,CALNU_0040_20241108_1,11/8/24,14:12,551174,4269175,26910,13.16,11.3,8,230
139,159,41_pre_puc_burn3d,burn3d,Burnpro3d,puc,Pacific Union College (PUC),pre,Prefire,41,CALNU_0041_20241108_1,11/8/24,12:41,551260,4269177,26910,13.16,11.3,58,69
140,160,45_pre_puc_burn3d,burn3d,Burnpro3d,puc,Pacific Union College (PUC),pre,Prefire,45,CALNU_0045_20241108_1,11/8/24,7:50,551220,4269331,26910,13.16,11.3,51,64
141,161,44_pre_puc_burn3d,burn3d,Burnpro3d,puc,Pacific Union College (PUC),pre,Prefire,44,CALNU_0044_20241107_1,11/7/24,14:15,551115,4269331,26910,13.16,11.3,11,304


In [10]:
fia_tree=pd.read_csv("FIA/CA_TREE.csv",low_memory=False)

In [11]:
fia_tree

,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,...,VOLCSNET_BARK,DRYBIO_STEM,DRYBIO_STEM_BARK,DRYBIO_STUMP_BARK,DRYBIO_BOLE_BARK,DRYBIO_BRANCH,DRYBIO_FOLIAGE,DRYBIO_SAWLOG_BARK,PREV_ACTUALHT_FLD,PREV_HT_FLD
0,675267,11839,NaN,1994,6,1,15,1,1,675267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,675268,11839,NaN,1994,6,1,15,1,1,675268,...,NaN,NaN,NaN,17.718832,469.383932,NaN,NaN,NaN,NaN,NaN
2,675269,11839,NaN,1994,6,1,15,1,1,675269,...,NaN,NaN,NaN,10.354492,143.559569,NaN,NaN,NaN,NaN,NaN
3,675270,11839,NaN,1994,6,1,15,1,1,675270,...,NaN,NaN,NaN,5.959414,68.141174,NaN,NaN,NaN,NaN,NaN
4,675271,11839,NaN,1994,6,1,15,1,1,675271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427514,1288119594290487,750083033290487,NaN,2021,6,6,73,77509,3,15,...,NaN,1.422900,0.386542,NaN,NaN,0.253708,0.787288,NaN,NaN,NaN
427515,1288119595290487,750083033290487,NaN,2021,6,6,73,77509,3,16,...,NaN,17.797914,3.597250,NaN,NaN,9.646469,9.169736,NaN,NaN,NaN
427516,1288119296290487,750082993290487,4.475655e+13,2021,6,6,73,87669,4,112,...,1.397503,195.318081,37.116506,5.689291,30.899558,226.160257,28.434277,23.236013,NaN,NaN
427517,1288119300290487,750082993290487,NaN,2021,6,6,73,87669,4,145,...,NaN,42.222643,8.760547,1.398266,4.988934,20.620192,5.873709,NaN,NaN,NaN


In [12]:
fia_tree.isnull().sum()

CN                         0
PLT_CN                     0
PREV_TRE_CN           200672
INVYR                      0
STATECD                    0
                       ...  
DRYBIO_BRANCH          89315
DRYBIO_FOLIAGE         80496
DRYBIO_SAWLOG_BARK    277175
PREV_ACTUALHT_FLD     427519
PREV_HT_FLD           427519
Length: 197, dtype: int64

In [13]:
fia_tree[fia_tree["SPCD"]>999]

,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,...,VOLCSNET_BARK,DRYBIO_STEM,DRYBIO_STEM_BARK,DRYBIO_STUMP_BARK,DRYBIO_BOLE_BARK,DRYBIO_BRANCH,DRYBIO_FOLIAGE,DRYBIO_SAWLOG_BARK,PREV_ACTUALHT_FLD,PREV_HT_FLD
375264,1123813164290487,558626624126144,NaN,2019,6,2,89,95748,1,1,...,NaN,199.255640,36.521290,4.564626,30.762931,155.188165,12.997852,NaN,NaN,NaN
375265,1123813165290487,558626624126144,NaN,2019,6,2,89,95748,1,3,...,NaN,152.498928,29.315211,4.016227,24.086692,128.136732,11.510544,NaN,NaN,NaN
375266,1123813166290487,558626624126144,NaN,2019,6,2,89,95748,1,4,...,NaN,55.940601,11.681241,1.915642,7.855255,40.283725,5.383091,NaN,NaN,NaN
405688,850381796290487,645307382126144,NaN,2020,6,5,47,89937,2,1,...,NaN,122.864642,18.323361,1.426713,13.102218,23.779039,4.327630,NaN,NaN,NaN
405689,850381797290487,645307382126144,NaN,2020,6,5,47,89937,2,2,...,NaN,96.950714,14.809717,1.213141,9.537884,18.500117,3.681699,NaN,NaN,NaN
405690,850381799290487,645307382126144,NaN,2020,6,5,47,89937,2,4,...,NaN,72.165834,13.507200,1.596235,9.075295,27.650948,4.489556,NaN,NaN,NaN
405691,850381800290487,645307382126144,NaN,2020,6,5,47,89937,2,5,...,NaN,91.460671,14.094829,1.175108,8.824781,17.597703,3.560234,NaN,NaN,NaN
405692,850381801290487,645307382126144,NaN,2020,6,5,47,89937,2,11,...,NaN,112.469052,16.901159,1.337963,11.656890,21.535484,4.062588,NaN,NaN,NaN
405693,850381802290487,645307382126144,NaN,2020,6,5,47,89937,2,12,...,NaN,122.864642,18.323361,1.426713,13.102218,23.779039,4.327630,NaN,NaN,NaN
405694,850381803290487,645307382126144,NaN,2020,6,5,47,89937,2,13,...,NaN,122.864642,18.323361,1.426713,13.102218,23.779039,4.327630,NaN,NaN,NaN


In [14]:
final = fia_tree.merge(fia_species,how='left',on='SPCD')

In [15]:
final["PFT"].value_counts()

PFT
Evergreen conifer          279767
Evergreen broadleaf         57704
Deciduous broadleaf         49594
Deciduous                   33760
Evergreen                    6431
Flowering plants              230
Unknown                         6
Broadleaf                       4
Nitrogen fixing legumes         2
Deciduous confier               2
Name: count, dtype: int64

In [17]:
final.isnull().sum().sort_values(ascending=False).head(100)

SAWHT                 427519
CULLDEAD              427519
CFSND                 427519
BFSND                 427519
CULLCF                427519
                       ...  
SEVERITY2_CD_PNWRS    385108
DAMLOC2               383325
DECAYCD               382868
RECONCILECD           379716
HRDWD_CLUMP_CD        375350
Length: 100, dtype: int64

In [18]:
final.shape

(427519, 208)

In [19]:
# DBH
final["DIA"] = final["DIA"] * 2.54

In [20]:
final["HT"] 

0         43.0
1         95.0
2         56.0
3         52.0
4         41.0
          ... 
427514    13.0
427515    18.0
427516    21.0
427517    20.0
427518    38.0
Name: HT, Length: 427519, dtype: float64

In [21]:
final["PFT"]

0                   Deciduous
1           Evergreen conifer
2                   Deciduous
3                   Deciduous
4           Evergreen conifer
                 ...         
427514      Evergreen conifer
427515      Evergreen conifer
427516    Evergreen broadleaf
427517    Evergreen broadleaf
427518    Evergreen broadleaf
Name: PFT, Length: 427519, dtype: object

In [22]:
final['DIA'].fillna(final['DIA'].median(), inplace=True)
final['HT'].fillna(final['HT'].median(), inplace=True)

/tmp/ipykernel_142/3706231944.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final['DIA'].fillna(final['DIA'].median(), inplace=True)


In [23]:
threshold = 0.2
final=final.loc[:, final.isnull().mean() < threshold]

In [24]:
final.isnull().sum().sort_values(ascending=False).head(20)

STOCKING          84116
CR                83153
CCLCD             83153
CULL              83141
DRYBIO_FOLIAGE    80496
VOLTSSND          79242
VOLTSGRS          79242
DIACHECK_PNWRS    79191
DIACHECK          79137
ACTUALHT          77822
SUBCYCLE          40685
DRYBIO_AG         38582
CARBON_AG         38560
DRYBIO_BG         38457
CARBON_BG         38457
TPA_UNADJ         38452
TREECLCD          38452
HTCD              38451
DIAHTCD            1381
OCC.NC               19
dtype: int64

In [25]:
final

,CN,PLT_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,CONDID,...,SCI_NAME,SPGRPCD.E,SPGRPCD.W,MAJGRP,OCC.NC,OCC.NE,OCC.PNW,OCC.RM,OCC.SO,PFT
0,675267,11839,1994,6,1,15,1,1,675267,1,...,Lithocarpus densiflorus,43.0,47.0,4.0,-,-,X,-,-,Deciduous
1,675268,11839,1994,6,1,15,1,1,675268,1,...,Pseudotsuga menziesii,9.0,10.0,2.0,X,X,X,X,X,Evergreen conifer
2,675269,11839,1994,6,1,15,1,1,675269,1,...,Lithocarpus densiflorus,43.0,47.0,4.0,-,-,X,-,-,Deciduous
3,675270,11839,1994,6,1,15,1,1,675270,1,...,Lithocarpus densiflorus,43.0,47.0,4.0,-,-,X,-,-,Deciduous
4,675271,11839,1994,6,1,15,1,1,675271,1,...,Pseudotsuga menziesii,9.0,10.0,2.0,X,X,X,X,X,Evergreen conifer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427514,1288119594290487,750083033290487,2021,6,6,73,77509,3,15,2,...,Pinus coulteri,9.0,24.0,1.0,-,-,X,-,-,Evergreen conifer
427515,1288119595290487,750083033290487,2021,6,6,73,77509,3,16,2,...,Pinus coulteri,9.0,24.0,1.0,-,-,X,-,-,Evergreen conifer
427516,1288119296290487,750082993290487,2021,6,6,73,87669,4,112,3,...,Quercus agrifolia,43.0,46.0,4.0,-,-,X,-,-,Evergreen broadleaf
427517,1288119300290487,750082993290487,2021,6,6,73,87669,4,145,3,...,Quercus agrifolia,43.0,46.0,4.0,-,-,X,-,-,Evergreen broadleaf


In [26]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427519 entries, 0 to 427518
Data columns (total 48 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CN              427519 non-null  int64  
 1   PLT_CN          427519 non-null  int64  
 2   INVYR           427519 non-null  int64  
 3   STATECD         427519 non-null  int64  
 4   UNITCD          427519 non-null  int64  
 5   COUNTYCD        427519 non-null  int64  
 6   PLOT            427519 non-null  int64  
 7   SUBP            427519 non-null  int64  
 8   TREE            427519 non-null  int64  
 9   CONDID          427519 non-null  int64  
 10  STATUSCD        427519 non-null  int64  
 11  SPCD            427519 non-null  float64
 12  SPGRPCD         427519 non-null  int64  
 13  DIA             427519 non-null  float64
 14  DIAHTCD         426138 non-null  float64
 15  HT              427519 non-null  float64
 16  HTCD            389068 non-null  float64
 17  ACTUALHT  

In [27]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427519 entries, 0 to 427518
Data columns (total 48 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CN              427519 non-null  int64  
 1   PLT_CN          427519 non-null  int64  
 2   INVYR           427519 non-null  int64  
 3   STATECD         427519 non-null  int64  
 4   UNITCD          427519 non-null  int64  
 5   COUNTYCD        427519 non-null  int64  
 6   PLOT            427519 non-null  int64  
 7   SUBP            427519 non-null  int64  
 8   TREE            427519 non-null  int64  
 9   CONDID          427519 non-null  int64  
 10  STATUSCD        427519 non-null  int64  
 11  SPCD            427519 non-null  float64
 12  SPGRPCD         427519 non-null  int64  
 13  DIA             427519 non-null  float64
 14  DIAHTCD         426138 non-null  float64
 15  HT              427519 non-null  float64
 16  HTCD            389068 non-null  float64
 17  ACTUALHT  

In [28]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427519 entries, 0 to 427518
Data columns (total 48 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CN              427519 non-null  int64  
 1   PLT_CN          427519 non-null  int64  
 2   INVYR           427519 non-null  int64  
 3   STATECD         427519 non-null  int64  
 4   UNITCD          427519 non-null  int64  
 5   COUNTYCD        427519 non-null  int64  
 6   PLOT            427519 non-null  int64  
 7   SUBP            427519 non-null  int64  
 8   TREE            427519 non-null  int64  
 9   CONDID          427519 non-null  int64  
 10  STATUSCD        427519 non-null  int64  
 11  SPCD            427519 non-null  float64
 12  SPGRPCD         427519 non-null  int64  
 13  DIA             427519 non-null  float64
 14  DIAHTCD         426138 non-null  float64
 15  HT              427519 non-null  float64
 16  HTCD            389068 non-null  float64
 17  ACTUALHT  

In [44]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 183.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 218.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


0         43.0
1         95.0
2         56.0
3         52.0
4         41.0
          ... 
427514    13.0
427515    18.0
427516    21.0
427517    20.0
427518    38.0
Name: HT, Length: 427519, dtype: float64

In [29]:

final['HT'] = np.log1p(final['HT'])  # log1p(x) = log(1 + x), safe for zero
final['DIA'] = np.log1p(final['DIA']) 

In [67]:
# Import necessary libraries
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Label Encoding for object columns
label_encoders = {}
object_columns = final.select_dtypes(include=['object']).columns

for col in object_columns:
    le = LabelEncoder()
    final[col] = le.fit_transform(final[col].astype(str))
    label_encoders[col] = le

# Define features (X) and target (Y)
X = final[["HT", "DIA"]]   # All columns except 'PFT'
Y = final['PFT']  # Target column

# Split data into train and test sets (80-20 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [31]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [68]:
# Required imports

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tabulate import tabulate
import numpy as np

# Define ML models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(eval_metric='mlogloss')
}

# Initialize results dictionary
results = {}

# Function to print results table
def print_results_table(results):
    headers = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
    
    rows = []
    for name, data in results.items():
        accuracy = data['accuracy']
        report = data['classification_report']['weighted avg']
        precision = report['precision']
        recall = report['recall']
        f1_score = report['f1-score']
        
        rows.append([name, accuracy, precision, recall, f1_score])
    
    print(tabulate(rows, headers=headers, tablefmt="pretty"))
    
    # Print confusion matrices separately
    for name, data in results.items():
        print(f"\nConfusion Matrix for {name}:")
        print(data['confusion_matrix'])

# Train and evaluate each model
for name, model in models.items():
    print(f"Training: {name}")
    model.fit(X_train, Y_train)
    
    print(f"Testing: {name}")
    Y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(Y_test, Y_pred)
    report = classification_report(Y_test, Y_pred, output_dict=True)
    cm = confusion_matrix(Y_test, Y_pred)
    
    results[name] = {
        'accuracy': accuracy,
        'classification_report': report,
        'confusion_matrix': cm
    }
    
    print(f"\nResults after adding {name}:")
    print_results_table(results)


Training: Logistic Regression


/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

Testing: Logistic Regression

Results after adding Logistic Regression:
+---------------------+--------------------+--------------------+--------------------+--------------------+
|        Model        |      Accuracy      |     Precision      |       Recall       |      F1-Score      |
+---------------------+--------------------+--------------------+--------------------+--------------------+
| Logistic Regression | 0.6399934505988024 | 0.4821359719300652 | 0.6399934505988024 | 0.5299725392485742 |
+---------------------+--------------------+--------------------+--------------------+--------------------+

Confusion Matrix for Logistic Regression:
[[    0     0     0     0     0     0     1     0     0     0]
 [    0   410     0     0     0   183  6142     0     0     0]
 [    0   210     0     0     0   307  9417     0     0     0]
 [    0     0     0     0     0     0     1     0     0     0]
 [    0    81     0     0     0    29  1140     0     0     0]
 [    0   167     0     0     

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing: Random Forest


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Results after adding Random Forest:
+---------------------+--------------------+--------------------+--------------------+--------------------+
|        Model        |      Accuracy      |     Precision      |       Recall       |      F1-Score      |
+---------------------+--------------------+--------------------+--------------------+--------------------+
| Logistic Regression | 0.6399934505988024 | 0.4821359719300652 | 0.6399934505988024 | 0.5299725392485742 |
|    Decision Tree    | 0.6329060628742516 | 0.5195353173988959 | 0.6329060628742516 | 0.5443110871633109 |
|    Random Forest    | 0.6410109468562875 | 0.5236627614153293 | 0.6410109468562875 | 0.5437546856790373 |
+---------------------+--------------------+--------------------+--------------------+--------------------+

Confusion Matrix for Logistic Regression:
[[    0     0     0     0     0     0     1     0     0     0]
 [    0   410     0     0     0   183  6142     0     0     0]
 [    0   210     0     0     0   307 

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Results after adding KNN:
+---------------------+--------------------+--------------------+--------------------+--------------------+
|        Model        |      Accuracy      |     Precision      |       Recall       |      F1-Score      |
+---------------------+--------------------+--------------------+--------------------+--------------------+
| Logistic Regression | 0.6399934505988024 | 0.4821359719300652 | 0.6399934505988024 | 0.5299725392485742 |
|    Decision Tree    | 0.6329060628742516 | 0.5195353173988959 | 0.6329060628742516 | 0.5443110871633109 |
|    Random Forest    | 0.6410109468562875 | 0.5236627614153293 | 0.6410109468562875 | 0.5437546856790373 |
|         KNN         | 0.5856100299401198 | 0.5271314006617356 | 0.5856100299401198 | 0.5510830306316362 |
+---------------------+--------------------+--------------------+--------------------+--------------------+

Confusion Matrix for Logistic Regression:
[[    0     0     0     0     0     0     1     0     0     0]
 [ 

KeyboardInterrupt: 

In [32]:
!pip install lazypredict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 170.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 187.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 187.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 201.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 177.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 137.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 181.6 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.8.2
    Uninstalling blinker-1.8.2:
      Successfully uninstalled blinker-1.8.2


In [33]:
X 

,HT,DIA
0,3.784190,2.389313
1,4.564348,4.130580
2,4.043051,3.612754
3,3.970292,3.320349
4,3.737670,1.994700
...,...,...
427514,2.639057,1.516445
427515,2.944439,2.478385
427516,3.091042,3.526949
427517,3.044522,2.635766


In [35]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd


# Stratified split to preserve label balance
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Show results
print("Model performance comparison:")
print(models)


2025/04/04 23:37:18 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.8.0, but the installed version is 3.9.2. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2025/04/04 23:37:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/04/04 23:37:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/04/04 23:37:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/04/04 23:37:18 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2025/04/04 23:37:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for ten

Model performance comparison:
Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


In [87]:
# Show top 5 models by accuracy
print(models.sort_values(by="Accuracy", ascending=False).head())


Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


In [50]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tabulate import tabulate
import numpy as np

# Define ML models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(eval_metric='mlogloss')
}

# Number of folds for cross-validation
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Initialize results dictionary
results = {}

# Function to print results table
def print_results_table(results):
    headers = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
    
    rows = []
    for name, data in results.items():
        accuracy = np.mean(data['accuracy'])
        precision = np.mean(data['precision'])
        recall = np.mean(data['recall'])
        f1_score = np.mean(data['f1-score'])
        
        rows.append([name, accuracy, precision, recall, f1_score])
    
    print(tabulate(rows, headers=headers, tablefmt="pretty"))

# Train and evaluate each model using K-Fold CV
for name, model in models.items():
    print(f"\nTraining & Evaluating: {name}")
    
    accuracies, precisions, recalls, f1_scores = [], [], [], []
    
    for train_index, test_index in skf.split(X, Y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)

        accuracy = accuracy_score(Y_test, Y_pred)
        report = classification_report(Y_test, Y_pred, output_dict=True)
        
        accuracies.append(accuracy)
        precisions.append(report['weighted avg']['precision'])
        recalls.append(report['weighted avg']['recall'])
        f1_scores.append(report['weighted avg']['f1-score'])
    
    results[name] = {
        'accuracy': accuracies,
        'precision': precisions,
        'recall': recalls,
        'f1-score': f1_scores
    }

# Print final results table
print("\nFinal Results Across All Models:")
print_results_table(results)



Training & Evaluating: Logistic Regression


/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}


Training & Evaluating: Decision Tree


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/m


Training & Evaluating: Random Forest


...

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/m


Training & Evaluating: XGBoost


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/m


Final Results Across All Models:
+---------------------+--------------------+--------------------+--------------------+--------------------+
|        Model        |      Accuracy      |     Precision      |       Recall       |      F1-Score      |
+---------------------+--------------------+--------------------+--------------------+--------------------+
| Logistic Regression | 0.6532809087974711 | 0.4291527389164102 | 0.6532809087974711 | 0.5173647573651975 |
|    Decision Tree    | 0.6340887769444182 |  0.52086964520066  | 0.6340887769444182 | 0.5447727274453836 |
|    Random Forest    | 0.6419036344546267 | 0.5257629677914617 | 0.6419036344546267 | 0.5449029524929061 |
|         KNN         | 0.5636498293236691 | 0.5318162995779112 | 0.5636498293236691 | 0.5437058936089567 |
|  Gradient Boosting  | 0.6552901727804259 | 0.5325272668143783 | 0.6552901727804259 | 0.5297642098060971 |
|       XGBoost       | 0.6557065298414434 | 0.5564968928467472 | 0.6557065298414434 | 0.5315709604416

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define base models
logreg = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier(eval_metric='mlogloss')

# Create Voting Classifier (soft voting uses predicted probabilities)
ensemble_model = VotingClassifier(
    estimators=[
        ('lr', logreg),
        ('dt', dt),
        ('rf', rf),
        ('gb', gb),
        ('xgb', xgb)
    ],
    voting='soft'  # Use 'hard' for majority class voting
)

# Train ensemble model
ensemble_model.fit(X_train, Y_train)

# Predict on test set
Y_pred = ensemble_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
cm = confusion_matrix(Y_test, Y_pred)

# Output
print(f"Ensemble Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("Confusion Matrix:\n", cm)


/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ensemble Accuracy: 0.6530

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.35      0.05      0.08      6752
           2       0.28      0.01      0.02      9919
           3       0.00      0.00      0.00         1
           4       0.25      0.00      0.00      1286
           5       0.24      0.01      0.02     11541
           6       0.66      0.99      0.79     55953
           7       0.00      0.00      0.00        46
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         3

    accuracy                           0.65     85503
   macro avg       0.18      0.11      0.09     85503
weighted avg       0.53      0.65      0.53     85503

Confusion Matrix:
 [[    0     0     0     0     0     0     1     0     0     0]
 [    0   325    16     0     1    14  6396     0     0     0]
 [    0   158   124     0     3    99  9535  

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [61]:
!pip install lightgbm
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 240.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 222.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 211.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 195.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 216.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 209.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 227.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


In [63]:
from imblearn.ensemble import BalancedRandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Define base models with imbalance handling
base_learners = [
    ('brf', BalancedRandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(scale_pos_weight=class_weight_ratio, eval_metric='mlogloss')),
    ('lgb', LGBMClassifier(is_unbalance=True))    # ← missing comma here
    ('cat', CatBoostClassifier(verbose=0, auto_class_weights='Balanced'))
]

from sklearn.linear_model import LogisticRegression
meta_learner = LogisticRegression(class_weight='balanced')
from sklearn.ensemble import StackingClassifier

# Stacking model
stacking_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    passthrough=True,
    cv=1
)

# Fit model
stacking_model.fit(X_train, Y_train)

# Evaluate
Y_pred = stacking_model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:\n", classification_report(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))


<>:10: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:10: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject